In [1]:
from hw2skeleton import io, cluster, utils
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from itertools import product
import pandas as pd
from sklearn.decomposition import PCA

In [2]:
#path = '/Users/Sisi/Documents/GitHub/BMI-203-HW2/data' #Note: may have to change the path when uploading into GitHub?
path = '/Users/sierra.lear/Documents/GitHub/BMI-203-HW2/data'

active_site = io.read_active_sites(path);

Read in 136 active sites


In [ ]:
active_site[0].residues

In [ ]:
active_site[3].residues[0].type

In [ ]:
list_res = []
for r in active_site[0].residues:
    list_res.append(r.type)
list_res

In [ ]:
list_res_2 = [r.type for r in active_site[0].residues]
list_res_2

## Objective #1:
*Implement a similarity metric to compare active sites.*

For my similarity metric, I want to calculate the number of a given amino acid in an active site. I can then calculate the Euclidan distance based on the 20-D space constructed by the ratio of all 20 different amino acids.

One one hand, this obviously simplifies or "ignores" important functional information, namely the 3D structure/configuration of amino acids or residues in relation to each other as well as the number of residues or how "large" an active site is.

However, given my inexperience with enzyme function, I am hypothesizing that the overall "chemical identity" of a residue still gives an important clue to its function--in particular, two active sites that contain more of the same amino acids are more likely to be functionally similar or have similar roles than two residues that contain no overlapping amino acids. Furthermore, given that some amino acids have specific charges or hydrophobicity, I know that amino acids often drive structure, so I also hypothesize that some of this "lost" structural information will actually be redundant or contained within the ratio or identity of amino acids anyway.

In [ ]:
len(active_site)

In [ ]:
combos = list(product(range(0,len(active_site)), range(0,len(active_site))))
combos_test =list(product(range(0,len(active_site)), range(0,len(active_site))))

In [98]:
my_aa = ['ALA', 'ARG', 'ASN', 'ASP', 'CYS', 'GLN', 'GLU', 'GLY', 'HIS', 'ILE', 'LEU', 'LYS',
         'MET', 'PHE', 'PRO', 'SER', 'THR', 'TRP', 'TYR', 'VAL']

def compute_similarity_partitioning(site_a, centroid):
    """
    Compute the similarity between two given ActiveSite instances.

    Input: one ActiveSite instance, one "centroid" consisting of a 20-element vector with numnbers between 0 and 1
    Output: the similarity between them (a floating point number)
    """

    similarity = 0.0
    
    #creating list comprehension of all AAs for site a residues
    list_res_a = [r.type for r in site_a.residues]
    
    #create two histogram count lists for site a and b from the list comprehensions
    count_a = Counter(list_res_a)
    
    #initialize two 20-element AA dictionaries: one for site A, one for site B
    a_dict = {aa:0 for aa in my_aa}
    
    #convert normalized histogram count into the 20-D vector AA dictionary
    #a site
    for aa, count in count_a.items():
        a_dict[aa] = count
    a_vector = np.array(list(a_dict.values()))
    a_vector = a_vector/np.sum(a_vector) #to return percentage of amino acid identity
        
    #calculate Euclidian distance between the two sites' 20-D vectors
    similarity = np.sqrt(np.sum((a_vector - centroid)**2))
    
    return similarity

In [ ]:
my_aa = ['ALA', 'ARG', 'ASN', 'ASP', 'CYS', 'GLN', 'GLU', 'GLY', 'HIS', 'ILE', 'LEU', 'LYS',
         'MET', 'PHE', 'PRO', 'SER', 'THR', 'TRP', 'TYR', 'VAL']

def compute_similarity(site_a, site_b):
    """
    Compute the similarity between two given ActiveSite instances.

    Input: two ActiveSite instances
    Output: the similarity between them (a floating point number)
    """

    similarity = 0.0
    
    #creating list comprehension of all AAs for site a residues
    list_res_a = [r.type for r in site_a.residues]
    
    #creating list comprehension of all AAs for site b residues
    list_res_b = [r.type for r in site_b.residues]
    
    #create two histogram count lists for site a and b from the list comprehensions
    count_a = Counter(list_res_a)
    count_b = Counter(list_res_b)
    
    #initialize two 20-element AA dictionaries: one for site A, one for site B
    a_dict = {aa:0 for aa in my_aa}
    b_dict = {aa:0 for aa in my_aa}
    
    #convert normalized histogram count into the 20-D vector AA dictionary
    #a site
    for aa, count in count_a.items():
        a_dict[aa] = count
    a_vector = np.array(list(a_dict.values()))
    a_vector = a_vector/np.sum(a_vector) #to return percentage of amino acid identity
    #b site
    for aa, count in count_b.items():
        b_dict[aa] = count
    b_vector = np.array(list(b_dict.values()))
    b_vector = b_vector/np.sum(b_vector)
        
    #calculate Euclidian distance between the two sites' 20-D vectors
    similarity = np.sqrt(np.sum((a_vector - b_vector)**2))
    
    return similarity

In [ ]:
compute_similarity(active_site[3], active_site[4]).shape

In [ ]:
df = pd.DataFrame(columns = ['Site_A', 'Site_B', 'Similarity'])
for a, b in combos:
    similarity = compute_similarity(active_site[a], active_site[b])
    df = df.append({'Site_A': a, 'Site_B': b, 'Similarity': similarity}, ignore_index=True)
df.head()

In [ ]:
df = df.loc[~(df["Site_A"] == df["Site_B"]), :] #remove all rows where Site_A == Site_B
df.tail()

In [ ]:
plt.figure()
plt.plot(x,y)
plt.show()

In [ ]:
sns.distplot(df["Similarity"], bins=50, norm_hist=False, kde=False)

In [ ]:
f, ax_me = plt.subplots()
sns.distplot(df["Similarity"], bins=50, norm_hist=False, kde=False, ax=ax_me)
ax_me.set(ylim=[0,300])
plt.show()

## Objective #2: 
*Implement a partitioning algorithm to cluster the set of active sites.* 


In [ ]:
vector = pd.DataFrame(index = '', columns = ['Site_A', 'Site_B', 'Similarity'])

In [ ]:
active_site[0].name

In [ ]:
pd.MultiIndex.from_product([['a.name'], [0], np.arange(0,20)], 
                                      names=['original_id', 'sequential_id', 'amino_acid_ID'])

In [ ]:
output_df = pd.DataFrame()

for i, a in enumerate(active_site):
    list_res = [r.type for r in a.residues]
    count_aa = Counter(list_res)
    aa_dict = {aa:0 for aa in my_aa}
    for aa, count in count_aa.items():
        aa_dict[aa] = count
    aa_vec = np.array(list(aa_dict.values()))
    midx = pd.MultiIndex.from_product([[i], np.arange(0,len(aa_vec))], 
                                      names=['sequential_id', 'amino_acid_ID'])
    temp_df = pd.DataFrame({'aa_counts' : aa_vec}, index=midx)
    output_df = output_df.append(temp_df)

In [ ]:
idx_slicer = pd.IndexSlice

In [ ]:
output_df.info()

In [ ]:
output_df.loc[idx_slicer[0,:],:]

In [ ]:
output_df.unstack('sequential_id').T.apply(lambda x: x - x.mean(), axis = 1).to_numpy()

In [ ]:
output_df.drop().unstack('original_id')

In [ ]:
  
    #creating list comprehension of all AAs for site a residues
    list_res_a = [r.type for r in site_a.residues]
    
    #creating list comprehension of all AAs for site b residues
    list_res_b = [r.type for r in site_b.residues]
    
    #create two histogram count lists for site a and b from the list comprehensions
    count_a = Counter(list_res_a)
    count_b = Counter(list_res_b)
    
    #initialize two 20-element AA dictionaries: one for site A, one for site B
    a_dict = {aa:0 for aa in my_aa}
    b_dict = {aa:0 for aa in my_aa}
    
    #convert histogram count into the 20-D vector AA dictionary
    #a site
    for aa, count in count_a.items():
        a_dict[aa] = count
    a_vector = np.array(list(a_dict.values()))
    #b site
    for aa, count in count_b.items():
        b_dict[aa] = count
    b_vector = np.array(list(b_dict.values()))

In [ ]:
active_site[3].residues

In [134]:
def cluster_by_partitioning(active_sites):
    """
    Cluster a given set of ActiveSite instances using a partitioning method.
    Input: a list of ActiveSite instances
    Output: a clustering of ActiveSite instances
            (this is really a list of clusters, each of which is list of
            ActiveSite instances)
    """

    #number of clusters
    k = 5 #based off histogram--while an elbow plot would be a better way to pick clusters, I chose
              #this for the sake of brevity
    
    #creating random centroids
        # my centroid will be a 20-element vector with randomized numbers between 0 and 1
    np.random.seed(5)
    centroid_list = [None] * k #initializing my list of k centroids
    for i in range(0,len(centroid_list)): #for loop to create and store all my centroids in my centroid_list
        centroid_list[i] = np.random.rand(20) #creating random numbers between 0 and 1 to fill in the 20 elements of each centroid vector
    
    
    #k-means algorithm    
    partition_combos = list(product(range(0,len(active_site)), range(0,len(centroid_list)))) #create list of tuples
                                                                                          #with all combinations of
                                                                                #active site instances and centroids
        
    df = pd.DataFrame(columns = ['ActiveSite', 'Centroids', 'Similarity'])
    for a, b in partition_combos:
        similarity = compute_similarity_partitioning(active_site[a], centroid_list[b])
        df = df.append({'ActiveSite': a, 'Centroids': b, 'Similarity': similarity}, ignore_index=True)
        df = df.reset_index()
    
    #labeling each ActiveSite to a centroid based on its similarity score
        #groupby ActiveSite and take minimum distance --> delete all the rows that don't have the min distance,
        #thus I'm only keeping the cells with the "labeled centroid" for each ActiveSite instance
        sorted_df = df.sort_values("Similarity").groupby(["ActiveSite"]).first()
        
    
    #Move centroid to the center of the cluster based on the mean
        #groupby centroid
    
    
    
    
    
    
    
    
    #     #create df containing "20-element vectors" based on number of each amino acid in a given ActiveSite instance
#     aav_df = pd.DataFrame() #initialize df

#     for i, a in enumerate(active_site): #looping through all 136 active sites
#         list_res = [r.type for r in a.residues] #creating list comprehension of all AAs for an active site
#         count_aa = Counter(list_res) #create histogram count list for active site from the list comprehensions
#         aa_dict = {aa:0 for aa in my_aa} #initialize 20-element AA dictionary
#         for aa, count in count_aa.items(): #convert histogram count into the 20-D vector AA dictionary
#             aa_dict[aa] = count
#         aa_vec = np.array(list(aa_dict.values()))
#         midx = pd.MultiIndex.from_product([[i], np.arange(0,len(aa_vec))], #create multiindex for final df
#                                       names=['sequential_id', 'amino_acid_ID'])
#         temp_df = pd.DataFrame({'aa_counts' : aa_vec}, index=midx)
#         aav_df = aav_df.append(temp_df)
#     aav_df = aav_df.unstack('sequential_id').T #pivot to create final matrix, tranposed so fits orientation of PCA in scikitlearn
#     aav_df = aav_df.apply(lambda x: ((x - x.mean())/np.var(x,ddof=1)), axis = 1)
    
    #perform PCA to further reduce points from 20-D space to 2-D space
 #   pca = PCA(n_components=2)
 #   proj_aa = pca.fit_transform(aav_df.to_numpy())
    
    #number of clusters
 #   k = 4 #picking 4 because histogram of similarity scores above shows 4 "humps"
    
    #picking random coordinates
    #coord_x = np.random.rand(0, np.max(), size = k) #random x coordinate
    

    return sorted_df

In [135]:
cluster = cluster_by_partitioning(active_site)

ValueError: cannot insert level_0, already exists

In [133]:
cluster

,Centroids,Similarity
ActiveSite,,
0.0,0.0,2.167310
1.0,0.0,2.171325
2.0,0.0,2.181629
3.0,0.0,2.211896
4.0,0.0,2.201248
...,...,...
131.0,0.0,2.171325
132.0,0.0,2.228054
133.0,0.0,2.171325


In [122]:
hm

'Similarity'

In [ ]:
sns.distplot(cluster["Similarity"], bins=50, norm_hist=False, kde=False)

In [ ]:
sns.scatterplot(cluster[:,0], cluster[:,1])

In [ ]:
# Number of clusters
k = 3
# X coordinates of random centroids
C_x = np.random.randint(0, np.max(X)-20, size=k)
# Y coordinates of random centroids
C_y = np.random.randint(0, np.max(X)-20, size=k)
C = np.array(list(zip(C_x, C_y)), dtype=np.float32)
print(C)

In [ ]:
np.array(list(save_this.values()))**2

In [ ]:
test_v = np.array(['ALA', 'ALA', 'GLY', 'PRO'])
test_hist, b = np.histogram(test_v, ['ALA', 'GLY', 'PRO'])

In [ ]:
my_dict = {'san francisco' : 'david',4 : 'sierra'}

In [ ]:
my_dict

In [ ]:
my_dict['kevin'] = 'ft collin'

In [ ]:
my_dict

In [ ]:
my_aa = ['ALA', 'ARG', 'ASN', 'ASP', 'CYS', 'GLN', 'GLU', 'GLY', 'HIS', 'ILE', 'LEU', 'LYS',
         'MET', 'PHE', 'PRO', 'SER', 'THR', 'TRP', 'TYR', 'VAL']
my_aa

In [ ]:
sample_aa_dict = {}
for aa in my_aa:
    sample_aa_dict[aa] = 0
print(sample_aa_dict)

In [ ]:
different_aa_dict = {aa:0 for aa in my_aa}
different_aa_dict

In [ ]:
from collections import Counter
test_v = np.array(['ALA', 'ALA', 'GLY', 'PRO'])
count_v = Counter(test_v)
sns.barplot(list(count_v.keys()), list(count_v.values()))

In [ ]:
for aa, count in count_v.items():
    sample_aa_dict[aa] = count
sample_aa_dict
sns.barplot(list(sample_aa_dict.keys()), list(sample_aa_dict.values()))

## Objective #3: 
*Implement an agglomerative algorithm to cluster the set of active sites.* 

In [ ]:
def cluster_hierarchically(active_sites):
    """
    Cluster the given set of ActiveSite instances using a hierarchical algorithm.                                                                  #

    Input: a list of ActiveSite instances
    Output: a list of clusterings
            (each clustering is a list of lists of Sequence objects)
    """

    # Create tuples listing every combination of active site with every other active site
    combos = list(product(range(0,len(active_site)), range(0,len(active_site))))
    
    #create data frame consisting of two active sites and the distance between them using my similarity metric
    df = pd.DataFrame(columns = ['Site_A', 'Site_B', 'Similarity'])
    for a, b in combos:
        similarity = compute_similarity(active_site[a], active_site[b])
        df = df.append({'Site_A': a, 'Site_B': b, 'Similarity': similarity}, ignore_index=True)
    df = df.loc[~(df["Site_A"] == df["Site_B"]), :] #remove all comparisons of an active site with itself
    
    #sort list of distances from lowest to highest distance (labelled "similarity") in df
    df = df.sort_values(by = ["Similarity"])

    return df.head()

In [35]:
def cluster_hierarchically(active_sites):
    """
    Cluster the given set of ActiveSite instances using a hierarchical algorithm.                                                                  #

    Input: a list of ActiveSite instances
    Output: a list of clusterings
            (each clustering is a list of lists of Sequence objects)
    """

    # Create tuples listing every combination of active site with every other active site
    combos = list(product(range(0,len(active_site_instance)), range(0,len(active_site_instance))))
    
    #create data frame consisting of two active sites and the distance between them using my similarity metric
    similarity_df = pd.DataFrame(columns = ['Site_A', 'Site_B', 'Similarity', "Cluster"])
    for i, tup in enumerate(combos):
        similarity = compute_similarity_test(active_site_instance.loc[combos[tup][0]], active_site_instance.loc[[tup][1]])
        similarity_df = similarity_df.append({'Site_A': a, 'Site_B': b, 'Similarity': similarity, "Cluster": i}, ignore_index=True)
        similarity_df = similarity_df.loc[~(similarity_df["Site_A"] == similarity_df["Site_B"]), :] #remove all rows where Site_A == Site_B  
    
    #sort list of distances from lowest to highest distance (labelled "similarity") in df
    similarity_df = similarity_df.sort_values(by = ["Similarity"])
    
#     cluster_0 = [] #initialize cluster list
#     #add two closest active site clusters to cluster list
#     merged.append(similarity_df.iloc[0, 0])
#     merged.append(similarity_df.iloc[0, 1])
    
 
        

    
    #need to delete duplicates in each cluster list at the end
    


    return similarity_df, merged



In [36]:
cluster, merge = cluster_hierarchically(active_site)

TypeError: list indices must be integers or slices, not tuple

In [23]:
cluster.head()

,Site_A,Site_B,Similarity
6085,45.0,10.0,0.0
14132,104.0,92.0,0.0
12693,94.0,3.0,0.0
18164,134.0,74.0,0.0
4955,36.0,96.0,0.0


In [24]:
merge

[45.0, 10.0]

In [30]:
counter = 0
merged_value(counter) = 5


SyntaxError: can't assign to function call (<ipython-input-30-65bfeda5e6be>, line 2)